## InternVL3.5 - 2B

In [1]:
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer, GenerationConfig

# Prepare generation config
gen_config = GenerationConfig(
    max_new_tokens=128,
    temperature=0.7,
    top_p=0.9
)

# Load tokenizer (needed for chat)
tokenizer = AutoTokenizer.from_pretrained("OpenGVLab/InternVL3_5-1B", trust_remote_code=True)

# Load model with FP16 and low CPU memory usage
model = AutoModel.from_pretrained(
    "OpenGVLab/InternVL3_5-2B",
    dtype=torch.float16,    # FP16 for Jetson GPU
    low_cpu_mem_usage=True,
    trust_remote_code=True, 
    device_map="auto"
).eval()  

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


FlashAttention2 is not installed.


In [2]:
next(model.parameters()).dtype

torch.float16

In [3]:
from torchvision import transforms

# Example: load a PIL image
img = Image.open("../tasks/bird.jpg").convert("RGB")

preprocess = transforms.Compose([
    transforms.Resize((384, 384)),   # match model expectation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])
img_tensor = preprocess(img).unsqueeze(0).half().to("cuda")


In [ ]:
# Test
question = "Describe this image"

# Inference with no_grad (saves memory, disables gradient tracking)
with torch.no_grad():
    output = model.chat(
        tokenizer=tokenizer,
        pixel_values=img_tensor,
        question=question,
        generation_config=gen_config.to_dict(),
        history=None
    )

print(output)
